In [8]:
from Gate import Transformer2
import torch
import numpy as np
from util import (get_tensors, get_mask, init_weights, get_std_opt)
import matplotlib.pyplot as plt
from util import (long_loss, surv_loss)
import pandas as pd

# Load the pickle file back into a DataFrame
data = pd.read_pickle('data/simulated_data.pkl')
I = data['id'].nunique()

        ## split train/test
random_id = range(I) #np.random.permutation(range(I))
train_id = random_id[0:int(0.7*I)]
test_id = random_id[int(0.7*I):I]

train_data = data[data["id"].isin(train_id)]
test_data = data[data["id"].isin(test_id)]

print(train_data.shape)

batch_long, batch_base, batch_mask, batch_e, batch_t, obs_time = get_tensors(train_data)


from Gate import Transformer2
from util import (get_tensors, get_mask, init_weights, get_std_opt)
model = Transformer2(d_long=3, d_base=2, d_model=32, nhead=4,
                    num_decoder_layers=4)
model.to('cuda')
model.apply(init_weights)
model = model.train()
optimizer = torch.optim.Adam(model.parameters(), lr=0, betas=(0.9, 0.98), eps=1e-9)
scheduler = get_std_opt(optimizer, d_model=32, warmup_steps=200, factor=0.2)

n_epoch = 25
batch_size = 32
    
    
loss_values = []

for epoch in range(n_epoch):
    running_loss = 0
    train_id = np.random.permutation(train_id)
    for batch in range(0, len(train_id), batch_size):
        optimizer.zero_grad()
            
        indices = train_id[batch:batch+batch_size]
        batch_data = train_data[train_data["id"].isin(indices)]
            
        batch_long, batch_base, batch_mask, batch_e, batch_t, obs_time = get_tensors(batch_data.copy())
        batch_long_inp = batch_long[:,:-1,:].to('cuda');batch_long_out = batch_long[:,1:,:].to('cuda')  #time 1-11 as train and 12 as validation 
        batch_base = batch_base[:,:-1,:].to('cuda')
        batch_mask_inp = get_mask(batch_mask[:,:-1]).to('cuda')
        batch_mask_out = batch_mask[:,1:].unsqueeze(2).to('cuda') 
        obs_time = obs_time.to('cuda')
        yhat_long, yhat_surv = model(batch_long_inp, batch_base, batch_mask_inp,
                        obs_time[:,:-1].to('cuda'), obs_time[:,1:].to('cuda'))
        
        loss1 = long_loss(yhat_long, batch_long_out, batch_mask_out)
        loss2 = surv_loss(yhat_surv, batch_mask, batch_e)
        
        loss = loss1 + loss2
        
        loss.backward()
        scheduler.step()
        running_loss += loss
    loss_values.append(running_loss.tolist())

(5329, 15)


/home/shijimao/Proj1/util.py:64: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:,"id_new"] = df.groupby(by="id").ngroup()


In [9]:
loss_values

[2043.4012317150646,
 1422.9890966364942,
 409.0796639388764,
 187.5692697760143,
 128.2551735082678,
 102.78405266641025,
 89.34673749983993,
 78.12358630860528,
 72.66275212371836,
 68.2862004447864,
 66.17905208508614,
 61.45592697641383,
 60.27143767313913,
 57.75243768603987,
 58.41671895790388,
 54.5605316917589,
 52.1290735161825,
 51.76978297910577,
 51.408707712024,
 49.792134436913216,
 49.4793501650751,
 49.46791411417364,
 47.88909201553294,
 48.553212971299224,
 47.317369462727534]

In [10]:
from Gate import Transformer1
from util import (get_tensors, get_mask, init_weights, get_std_opt)
model = Transformer1(d_long=3, d_base=2, d_model=32, nhead=4,
                    num_decoder_layers=4)
model.to('cuda')
model.apply(init_weights)
model = model.train()
optimizer = torch.optim.Adam(model.parameters(), lr=0, betas=(0.9, 0.98), eps=1e-9)
scheduler = get_std_opt(optimizer, d_model=32, warmup_steps=200, factor=0.2)

n_epoch = 25
batch_size = 32
    
    
loss_values = []

for epoch in range(n_epoch):
    running_loss = 0
    train_id = np.random.permutation(train_id)
    for batch in range(0, len(train_id), batch_size):
        optimizer.zero_grad()
            
        indices = train_id[batch:batch+batch_size]
        batch_data = train_data[train_data["id"].isin(indices)]
            
        batch_long, batch_base, batch_mask, batch_e, batch_t, obs_time = get_tensors(batch_data.copy())
        batch_long_inp = batch_long[:,:-1,:].to('cuda');batch_long_out = batch_long[:,1:,:].to('cuda')  #time 1-11 as train and 12 as validation 
        batch_base = batch_base[:,:-1,:].to('cuda')
        batch_mask_inp = get_mask(batch_mask[:,:-1]).to('cuda')
        batch_mask_out = batch_mask[:,1:].unsqueeze(2).to('cuda') 
        obs_time = obs_time.to('cuda')
        yhat_long, yhat_surv = model(batch_long_inp, batch_base, batch_mask_inp,
                        obs_time[:,:-1].to('cuda'), obs_time[:,1:].to('cuda'))
        
        loss1 = long_loss(yhat_long, batch_long_out, batch_mask_out)
        loss2 = surv_loss(yhat_surv, batch_mask, batch_e)
        
        loss = loss1 + loss2
        
        loss.backward()
        scheduler.step()
        running_loss += loss
    loss_values.append(running_loss.tolist())

loss_values

[1831.1320527013017,
 1199.998580859093,
 854.9263858985971,
 677.8987792280332,
 537.5691398058576,
 405.1028393656558,
 307.9335351009386,
 226.012744726704,
 154.3657622385552,
 114.45698637192794,
 93.54460879748446,
 81.4560243971994,
 78.60034991785948,
 72.51075039819777,
 70.64321157498858,
 66.34071773469091,
 64.76633948334279,
 63.34772855710717,
 59.86534780977418,
 57.44026712372543,
 59.553240030848805,
 56.28156074969739,
 57.48824605266946,
 57.96418045701528,
 55.07980394717762]

In [13]:
from Gate import Transformer3
from util import (get_tensors, get_mask, init_weights, get_std_opt)
model = Transformer3(d_long=3, d_base=2, d_model=32, nhead=4,
                    num_decoder_layers=4)
model.to('cuda')
model.apply(init_weights)
model = model.train()
optimizer = torch.optim.Adam(model.parameters(), lr=0, betas=(0.9, 0.98), eps=1e-9)
scheduler = get_std_opt(optimizer, d_model=32, warmup_steps=200, factor=0.2)

n_epoch = 25
batch_size = 32
    
    
loss_values = []

for epoch in range(n_epoch):
    running_loss = 0
    train_id = np.random.permutation(train_id)
    for batch in range(0, len(train_id), batch_size):
        optimizer.zero_grad()
            
        indices = train_id[batch:batch+batch_size]
        batch_data = train_data[train_data["id"].isin(indices)]
            
        batch_long, batch_base, batch_mask, batch_e, batch_t, obs_time = get_tensors(batch_data.copy())
        batch_long_inp = batch_long[:,:-1,:].to('cuda');batch_long_out = batch_long[:,1:,:].to('cuda')  #time 1-11 as train and 12 as validation 
        batch_base = batch_base[:,:-1,:].to('cuda')
        batch_mask_inp = get_mask(batch_mask[:,:-1]).to('cuda')
        batch_mask_out = batch_mask[:,1:].unsqueeze(2).to('cuda') 
        obs_time = obs_time.to('cuda')
        yhat_long, yhat_surv = model(batch_long_inp, batch_base, batch_mask_inp,
                        obs_time[:,:-1].to('cuda'), obs_time[:,1:].to('cuda'))
        
        loss1 = long_loss(yhat_long, batch_long_out, batch_mask_out)
        loss2 = surv_loss(yhat_surv, batch_mask, batch_e)
        
        loss = loss1 + loss2
        
        loss.backward()
        scheduler.step()
        running_loss += loss
    loss_values.append(running_loss.tolist())

loss_values

ImportError: cannot import name 'Transformer3' from 'Gate' (/home/shijimao/Proj1/Gate.py)